# Imports

In [1]:
from PyPDF2 import PdfFileReader
import re
from datetime import datetime
from pprint import pprint
import pandas as pd

# Functions

In [2]:
def scrape_page(page_num):
    page = pdf.getPage(page_num)
    page_content = page.extractText()
    message_num_re = re.match(r'^Message ([0-9]+) of [0-9]+\n', page_content)
    end = message_num_re.end()
    message_num = int(message_num_re.group(1))
    message_body_re = re.match(r'([\s\S]*)Sent: ', page_content[end:], flags=re.MULTILINE) 
    message_body = message_body_re.group(1)
    end = message_body_re.end() + end
    sent_date_re = re.match(r'([0-9]{2}/[0-9]{2}/[0-9]{4} at [0-9]{2}:[0-9]{2} (AM|PM))', page_content[end:], flags=re.MULTILINE) 
    sent_date = sent_date_re.group(1)
    dt_obj_sent_date = datetime.strptime(sent_date, '%m/%d/%Y at %I:%M %p')
    end = sent_date_re.end() + end
    from_name_re = re.match(r'\nFrom: (.*)$', page_content[end:], flags=re.MULTILINE)
    from_name = from_name_re.group(1)
    end = from_name_re.end() + end
    to_name_re = re.match(r'\nTo: (.*) \(First Viewed: ', page_content[end:], flags=re.MULTILINE)
    to_name = to_name_re.group(1)
    end = to_name_re.end() + end
    first_viewed_re = re.match(r'([0-9]{2}/[0-9]{2}/[0-9]{4} at [0-9]{2}:[0-9]{2} (AM|PM))', page_content[end:], flags=re.MULTILINE)
    first_viewed = first_viewed_re.group(1)
    dt_obj_first_viewed = datetime.strptime(first_viewed, '%m/%d/%Y at %I:%M %p')
    next_re = re.match(r'([\s\S]*)', page_content[end:], flags=re.MULTILINE)
    end = first_viewed_re.end() + end
    subject_re = re.match(r'\)\nSubject: (.*)$', page_content[end:], flags=re.MULTILINE)
    subject = subject_re.group(1)
    _dict = {'page': page_num + 1, 
             'message_num': message_num,
             'message_body': message_body,
             'sent_date': dt_obj_sent_date,
             'from_name': from_name,
             'to_name': to_name,
             'first_viewed': dt_obj_first_viewed,
             'subject': subject,
             'is_reply': subject[0:3].upper() == 'RE:',
            }
    return _dict

def get_fullpage(page_num):
    page = pdf.getPage(page_num)
    page_content = page.extractText()
    message_re = re.match(r'([\s\S]*)Page', page_content, flags=re.MULTILINE) 
    message = message_re.group(1)
    return '\n' + message

# body

In [3]:
pdf = PdfFileReader('emails.pdf')
number_of_pages = pdf.getNumPages()
information = pdf.getDocumentInfo()

emails = []

page = 0
while True:
    try:
        emails.append(scrape_page(page))
    except AttributeError:
        last_email = emails[-1]
        last_email['message_body'] = last_email['message_body'] + get_fullpage(page)
    except IndexError:
        break
    page += 1

In [ ]:
df = pd.DataFrame(emails)


In [392]:
df.to_csv('emails.csv')